In [50]:
%load_ext autoreload
%autoreload 2
from histopolalign.VerificationAlignment import verify_alignment_procedure

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
verify_alignment_procedure.verify_alignment_main()

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\romai\\Documents\\PathologyPaper\\data\\TumorMeasurements\\59\\2022-11-02_T_HORAO-59-AF_FR_S2_1\\histology\\labels_GM_WM_original.png'

In [4]:
from tqdm.notebook import tqdm
dice_scores = {}

size_neighbors = 4

for folder in tqdm(os.listdir(data_folder_path)):
    image_gt_path = os.path.join(data_folder_path, folder, 'annotation', 'merged.png')
    image_gt = np.array(Image.open(image_gt_path))
    
    img_ROI = np.array(Image.open(os.path.join(data_folder_path, folder, 'annotation', 'ROI.tif')))
    
    image_labels_raw_path = os.path.join(data_folder_path, folder, 'histology', 'labels_GM_WM_original.png')
    image_labels_raw = np.array(Image.open(image_labels_raw_path))
    
    image_labels_init_path = os.path.join(data_folder_path, folder, 'histology', 'labels_GM_WM.png')
    image_labels_inti = np.array(Image.open(image_labels_init_path))
    
    image_labels_contour_path = os.path.join(data_folder_path, folder, 'histology', 'aligned' ,'histology_labels_GM_WM_contour.png')
    image_labels_contour = np.array(Image.open(image_labels_contour_path))
    
    image_labels_aligned_path = os.path.join(data_folder_path, folder, 'histology', 'aligned' ,'labels_GM_WM_aligned_MLS.png')
    image_labels_aligned = np.array(Image.open(image_labels_aligned_path))
    image_labels_aligned = create_image_labels_final(image_labels_aligned, colors = [[153, 153, 0], [153, 77, 0]])
    
    image_labels_final_path = os.path.join(data_folder_path, folder, 'histology', 'labels_augmented_GM_WM_masked.png')
    image_labels_final = np.array(Image.open(image_labels_final_path))
    image_labels_final = create_image_labels_final(image_labels_final, colors = [[153, 153, 0], [153, 77, 0]])
    
    
    dice_score = {'raw': 0, 'pre-align': 0, 'contour': 0, 'MLS': 0, 'final': 0}
    
    border_gt = get_border(image_gt, img_labels = False, size_neighbors = size_neighbors, colors = [128, 255])
    border_gt_img = Image.fromarray(border_gt.astype(np.uint8) * 255)
    border_gt_img.save(os.path.join(data_folder_path, folder, 'histology', 'border_ground_truth.png'))
    
    border_raw = get_border(image_labels_raw, img_labels = True, size_neighbors = size_neighbors, colors = [[153, 153, 0], [153, 77, 0]])
    border_raw_img = Image.fromarray(border_raw.astype(np.uint8) * 255)
    border_raw_img.save(os.path.join(data_folder_path, folder, 'histology', 'border_raw.png'))
    
    border_inti = get_border(image_labels_inti, img_labels = True, size_neighbors = size_neighbors, colors = [[153, 153, 0], [153, 77, 0]])
    border_inti_img = Image.fromarray(border_inti.astype(np.uint8) * 255)
    border_inti_img.save(os.path.join(data_folder_path, folder, 'histology', 'border_initialized.png'))
    
    border_contour = get_border(image_labels_contour, img_labels = True, size_neighbors = size_neighbors, colors = [[153, 153, 0], [153, 77, 0]])
    border_contour_img = Image.fromarray(border_contour.astype(np.uint8) * 255)
    border_contour_img.save(os.path.join(data_folder_path, folder, 'histology', 'border_contour.png'))
    
    border_aligned = get_border(image_labels_aligned, img_labels = True, size_neighbors = size_neighbors, colors = [[153, 153, 0], [153, 77, 0]])
    border_aligned_img = Image.fromarray(border_aligned.astype(np.uint8) * 255)
    border_aligned_img.save(os.path.join(data_folder_path, folder, 'histology', 'border_aligned.png'))
    
    border_final = get_border(image_labels_final, img_labels = True, size_neighbors = size_neighbors, colors = [[153, 153, 0], [153, 77, 0]])
    border_final_img = Image.fromarray(border_final.astype(np.uint8) * 255)
    border_final_img.save(os.path.join(data_folder_path, folder, 'histology', 'border_final.png'))
    
    img_reference = border_gt
    img_aligned = [border_raw, border_inti, border_contour, border_aligned, border_final]

    for img, (key,_) in zip(img_aligned, dice_score.items()):
        dice_score[key] = dice(img_reference, img, k = 1)
        overimposed = border_gt + img
        overimposed = Image.fromarray(overimposed.astype(np.uint8) * 120)
        overimposed.save(os.path.join(data_folder_path, folder, 'histology', 'overimposed_' + key + '.png'))
        
    dice_scores[folder] = dice_score

  0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
from collections import defaultdict
dice_avg = defaultdict(list)

In [6]:
for _, val in dice_scores.items():
    for key, dic in val.items():
        dice_avg[key].append(dic)

In [7]:
dice_mean = {}
dice_std = {}

for key, val in dice_avg.items():
    dice_mean[key] = np.mean(val)
    dice_std[key] = np.std(val)

In [8]:
dice_mean, dice_std

({'raw': 0.31395986822575567,
  'pre-align': 0.6523120944968757,
  'contour': 0.6634076470408482,
  'MLS': 0.7536704554140634,
  'final': 0.737978886993073},
 {'raw': 0.05916715716378708,
  'pre-align': 0.08239922605587097,
  'contour': 0.0863016444095431,
  'MLS': 0.04045879759025006,
  'final': 0.057175259486290264})